[The author's git-->](https://github.com/baranovxyz)

In [1]:
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='--packages com.databricks:spark-csv_2.10:1.2.0 pyspark-shell'
os.environ["PYSPARK_PYTHON"]='python3'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
print(spark_home)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.4-src.zip'))
os.environ["PYSPARK_PYTHON"] = 'python3'
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

/usr/hdp/current/spark2-client
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.3.2
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [1]:
import re
import pandas as pd
import numpy as np
import pickle
import json
import operator as op
from pyspark import AccumulatorParam
from pyspark.sql import Row
from pyspark.sql.functions import concat_ws, regexp_extract, split
import math

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
def row2prop(prop):
    return lambda row: row[prop]
def row2props(*props):
    return lambda row: [row[prop] for prop in props]

### Считаем tfidf по всем текстам

In [ ]:
df = spark.read.json("/labs/project2/item_details_full")

In [ ]:
re_word = re.compile('[^\W_\d]{3,}')

In [ ]:
dw = df.select(
    df.itemid.cast('integer'), 
    df.parent_id.cast('integer'), 
    concat_ws(' ', *['attr' + str(x) for x in range(78)]).alias('text'))

In [ ]:
dw.rdd.take(1)

In [ ]:
from nltk.corpus import stopwords
br_stop = sc.broadcast(set(stopwords.words('russian')))

In [ ]:
df_words = dw.rdd.map(lambda row: Row(
    words=[w for w in re_word.findall(row.text.lower() if row.text is not None else '')
           if w not in br_stop.value],
    **row.asDict())
).toDF()

In [ ]:
from pyspark.ml.feature import HashingTF
htf = HashingTF(numFeatures=1 << 16, binary=False, inputCol='words', outputCol='vec')
1 << 16 # в соответствии с рекомендациями из документации для числа хешей берем степень 2х

In [ ]:
df_bag = htf.transform(df_words)
df_bag.select('itemid','parent_id','words','vec')\
    .write.parquet("df_bag", "overwrite")

In [ ]:
df_bag = spark.read.parquet("df_bag")

In [ ]:
from pyspark.ml.feature import IDF
idf = IDF(minDocFreq=2, inputCol='vec', outputCol='tfidf')
md_idf = idf.fit(df_bag)

In [ ]:
md_idf.transform(df_bag)\
    .write.parquet("df_tfidf", "overwrite")

In [ ]:
#### Формируем разреженную матрицу из всех векторов

In [ ]:
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
df_bag = spark.read.parquet("df_bag")
# pseudo
csr_all = sparse.vstack((df_bag.collect()))
pickle.dump(csr_all, open('csr_all','xb'))

### Считаем векторную близость по всей выборке батчами

In [6]:
""" где то до этого распарсены данные по айдишникам и тд """
test_ids_w_index = pickle.load(open('test_ids_w_index','rb'))
csr_all = pickle.load(open('csr_all','rb'))
test_indexes = np.array([v for k,v in test_ids_w_index])

In [7]:
csr_all

<9273625x65536 sparse matrix of type '<class 'numpy.float64'>'
	with 450587057 stored elements in Compressed Sparse Row format>

In [6]:
test_ids_w_index[:3]

[(28759795, 2828040), (32712593, 6996810), (29616882, 2828041)]

In [ ]:
""" получить topn лучших совпадений по тфидф """
def get_best(csr1, csr2, topn):
    sim = cosine_similarity(csr1, csr2)
    ind = np.argpartition(sim, -topn, axis=1) # trick очень быстрый отбор топа индексов
    best_values = []
    best_indexes = []
    for i in range(sim.shape[0]):
        best_values.append(sim[i,ind[i,-topn:]])
        best_indexes.append(ind[i,-topn:])
    del sim
    del ind
    return (best_values, best_indexes)

In [ ]:
# лучше брать побольше кандидатов (topn)
batch_size = 200
topn = 200

In [ ]:
""" запускаем для 61000 то есть 305 итераций """
for i in range(math.ceil(len(test_indexes) / batch_size)):
    lower = i * batch_size
    upper = (i + 1) * batch_size
    if upper > len(test_indexes):
        upper = len(test_indexes)
    print(i)
    ts = datetime.timestamp(datetime.now())
    values_, indexes_ = get_best(csr_all[test_indexes[lower:upper],:],
                               csr_all,
                                topn)
    pickle.dump(indexes_, open('test/indexes_' + str(i),'xb'))
    del indexes_
    pickle.dump(values_, open('test/values_' + str(i),'xb'))
    del values_
    print(datetime.timestamp(datetime.now()) - ts)

In [ ]:
# где то здесь собираем все результаты в test_200

In [2]:
test_200 = pickle.load(open('test_200','rb'))
test_200[0]

(28759795,
 [(28759795, 1.0000000000000002),
  (28759802, 0.66001435046984847),
  (32010213, 0.55357484996739637),
  (28759787, 0.48544407543324869),
  (28759785, 0.45807386520829008),
  (30213185, 0.42517367435160303),
  (28759781, 0.42334206564127913),
  (19381453, 0.4144645645514341),
  (28759782, 0.38271865509362424),
  (6259845, 0.37913700830682201),
  (31413158, 0.37051680060386782),
  (28759803, 0.36447413609814838),
  (28759808, 0.36031454345894004),
  (28759783, 0.35875332577891361),
  (28759796, 0.33852610796613675),
  (28759809, 0.3307316945268397),
  (30213180, 0.32498457738439063),
  (18614210, 0.32003192850495332),
  (3897626, 0.31871808573955257),
  (30161483, 0.31526071119038213),
  (28759815, 0.31390572237856074),
  (28759801, 0.3062419264343984),
  (32010193, 0.29869872504693296),
  (4142391, 0.29567591922286751),
  (30213248, 0.29231267303480529),
  (28759780, 0.28856375589532679),
  (32677679, 0.28117082808283067),
  (22294338, 0.2781764857699473),
  (30213254, 0.27

## Расчитываем число хопов для кандидатов
* Предположительно сначала стоит рекомендовать айтемы из того же или ближайших каталогов

In [3]:
catalog_paths = pickle.load(open('catalog_paths','rb'))
catalog_ids = pickle.load(open('catalog_ids','rb'))

In [21]:
catalog_ids[29616882]

[1175232]

In [22]:
catalog_paths[1175232]

array([1175232, 1175180, 1159728, 1147991, 1132524], dtype=int64)

In [ ]:
def closest_path(item_id_1, item_id_2):
    try:
        catalogs_1 = catalog_ids[item_id_1]
        catalogs_2 = catalog_ids[item_id_2]
        if len(set(catalogs_1) & set(catalogs_2)) > 0:
            return 0
        catalog_paths_1 = [catalog_paths[n] for n in catalogs_1]
        catalog_paths_2 = [catalog_paths[n] for n in catalogs_2]
        all_lengths = []
        for path1 in catalog_paths_1:
            for path2 in catalog_paths_2:
                min_len = min(len(path1), len(path2))
                max_len = max(len(path1), len(path2))
                num_match = sum(path1[-min_len:] == path2[-min_len:]) # trick
                res = (max_len - min_len) + 2 * (min_len - num_match)
                if num_match == min_len:
                    return res
                all_lengths.append(res)
        return min(all_lengths)
    except:
        return -1

In [ ]:
def get_hops(recom_ids):
    hops = []
    for item_id, recoms in recom_ids:
        cur_hops = [closest_path(item_id, recom_id) for recom_id in recoms]
        hops.append((item_id, cur_hops))
    return hops

In [ ]:
test_hops = get_hops(recom_ids)

## Подбор параметров и сабмит
* Товары с высоким рейтингом должны получать плюс к мере сходства
* Чем дальше товар по структуре каталогов (или если не найден) тем больше штраф к сходству

In [27]:
ratings = pickle.load(open('ratings','rb'))
[t for t in ratings.items()][:3]

[(3212287, 5.0), (6466220, 5.0), (6291470, 3.0)]

In [4]:
test_hops = pickle.load(open('test_200_hops','rb'))
test_hops[:1]

[(28759795,
  [0,
   0,
   -1,
   0,
   0,
   0,
   0,
   0,
   0,
   3,
   0,
   0,
   0,
   0,
   0,
   2,
   0,
   -1,
   -1,
   0,
   2,
   0,
   -1,
   -1,
   0,
   0,
   -1,
   0,
   0,
   2,
   0,
   0,
   0,
   -1,
   2,
   -1,
   0,
   0,
   0,
   8,
   0,
   0,
   0,
   0,
   -1,
   -1,
   0,
   0,
   0,
   -1,
   5,
   -1,
   -1,
   8,
   -1,
   -1,
   0,
   2,
   2,
   0,
   2,
   9,
   0,
   7,
   0,
   0,
   2,
   0,
   -1,
   0,
   0,
   0,
   -1,
   2,
   2,
   2,
   2,
   -1,
   9,
   0,
   -1,
   0,
   0,
   0,
   2,
   3,
   0,
   3,
   -1,
   8,
   2,
   2,
   8,
   -1,
   0,
   0,
   -1,
   -1,
   0,
   -1,
   2,
   0,
   -1,
   -1,
   2,
   0,
   4,
   -1,
   0,
   7,
   -1,
   -1,
   0,
   -1,
   -1,
   2,
   2,
   2,
   8,
   8,
   8,
   2,
   0,
   0,
   3,
   -1,
   2,
   3,
   3,
   -1,
   -1,
   -1,
   10,
   3,
   2,
   -1,
   2,
   0,
   -1,
   0,
   2,
   9,
   0,
   0,
   0,
   -1,
   0,
   -1,
   0,
   3,
   0,
   -1,
   0,
   0,
   -1,
   2,
   0,
   0

In [33]:
a1 = 3 # коэффициент
a2 = 0.33 # оценка с которой начинаются "хорошие"
b = 0.1 # коффициент
def corr_rating(k,v,i,j):
    rating = ratings.get(int(k),-1)
    hops = test_hops[i][1][j]
    if hops == -1:
        hops = 7
    res = v if rating <= a1 else v * (1 + a2 * (rating * a1 - 1))
    res = res if hops == -1 else res * (1 - b * hops)
    return res

In [ ]:
result = {}
for i,predict in enumerate(test_200):
    item,recoms = predict
    item_id = str(int(item))
    result[item_id] = [(str(k), corr_rating(k,v,i,j)) for j,(k,v) in enumerate(recoms) if k!=item]

In [ ]:
result_top = {}
for item_id,recoms in result.items():
    result_top[item_id] = [(v,k) for k,v in sorted([(v,k) for k,v in recoms][:200], reverse=True)]

In [ ]:
with open('D:\\p2\\data\\ozon_test.txt','r') as rf:
    with open('submit','w') as wf:
        for line in rf:
            item = json.loads(line).get('item','-1')
            recoms = dict(result_top.get(item, []))
            wf.write(json.dumps({'item': str(item), 'recoms': recoms}) + '\n')

In [ ]:
!scp submit vb:project02.txt